# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [20]:
import napari
from octopuslite import utils, tile
import numpy as np

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
image_dir = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Images/'
metadata_fn = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Index.idx.xml'
metadata = utils.read_harmony_metadata(metadata_fn)

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


In [3]:
metadata

,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### Spatiotemporal scale 

In [6]:
metadata.columns

Index(['id', 'State', 'URL', 'Row', 'Col', 'FieldID', 'PlaneID', 'TimepointID',
       'ChannelID', 'FlimID', 'ChannelName', 'ImageType', 'AcquisitionType',
       'IlluminationType', 'ChannelType', 'ImageResolutionX',
       'ImageResolutionY', 'ImageSizeX', 'ImageSizeY', 'BinningX', 'BinningY',
       'MaxIntensity', 'CameraType', 'PositionX', 'PositionY', 'PositionZ',
       'AbsPositionZ', 'MeasurementTimeOffset', 'AbsTime',
       'MainExcitationWavelength', 'MainEmissionWavelength',
       'ObjectiveMagnification', 'ObjectiveNA', 'ExposureTime',
       'OrientationMatrix'],
      dtype='object')

In [7]:
from datetime import datetime

In [8]:
t1 = metadata[metadata['URL'] == 'r03c05f01p01-ch1sk1fk1fl1.tiff']['AbsTime']
metadata[metadata['URL'] == 'r03c05f01p01-ch1sk1fk1fl1.tiff']['AbsTime']

108    2021-04-16T19:11:18.61+01:00
Name: AbsTime, dtype: object

In [9]:
t3 = metadata[metadata['URL'] == 'r03c05f01p01-ch1sk2fk1fl1.tiff']['AbsTime']
metadata[metadata['URL'] == 'r03c05f01p01-ch1sk2fk1fl1.tiff']['AbsTime']

1620    2021-04-16T20:11:17.52+01:00
Name: AbsTime, dtype: object

In [10]:
t2 = metadata[metadata['URL'] == 'r03c05f01p01-ch1sk75fk1fl1.tiff']['AbsTime']
metadata[metadata['URL'] == 'r03c05f01p01-ch1sk75fk1fl1.tiff']['AbsTime']

111996    2021-04-19T21:11:19.17+01:00
Name: AbsTime, dtype: object

In [15]:
time_zone = str(t1.item()).split('+')[-1]

In [10]:
t1 = datetime.strptime(str(t1.item()), f"%Y-%m-%dT%H:%M:%S.%f+{time_zone}")

In [11]:
t2 = datetime.strptime(str(t2.item()), f"%Y-%m-%dT%H:%M:%S.%f+{time_zone}")

In [12]:
t3 = datetime.strptime(str(t3.item()), f"%Y-%m-%dT%H:%M:%S.%f+{time_zone}")

In [11]:
t1, t2, t3

(108    2021-04-16T19:11:18.61+01:00
 Name: AbsTime, dtype: object,
 111996    2021-04-19T21:11:19.17+01:00
 Name: AbsTime, dtype: object,
 1620    2021-04-16T20:11:17.52+01:00
 Name: AbsTime, dtype: object)

In [15]:
dt = t2 - t1
dt

datetime.timedelta(days=3, seconds=7200, microseconds=560000)

In [16]:
dt2 = t3-t1
dt2

datetime.timedelta(seconds=3598, microseconds=910000)

In [12]:
position_df = metadata[(metadata['Row'] == "3") 
                          & (metadata['Col'] == "5")
                          & (metadata['PlaneID'] == "1")
                          & (metadata['ChannelID'] == "1")
                          & (metadata['FieldID'] == "1")]

In [13]:
position_df

,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
108,0305K1F1P1R1,Ok,r03c05f01p01-ch1sk1fk1fl1.tiff,3,5,1,1,0,1,1,...,0,0.135621503,0,2021-04-16T19:11:18.61+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1620,0305K2F1P1R1,Ok,r03c05f01p01-ch1sk2fk1fl1.tiff,3,5,1,1,1,1,1,...,0,0.1356235,3598.91,2021-04-16T20:11:17.52+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3132,0305K3F1P1R1,Ok,r03c05f01p01-ch1sk3fk1fl1.tiff,3,5,1,1,2,1,1,...,0,0.135623902,7199.507,2021-04-16T21:11:18.117+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4644,0305K4F1P1R1,Ok,r03c05f01p01-ch1sk4fk1fl1.tiff,3,5,1,1,3,1,1,...,0,0.135622501,10800.82,2021-04-16T22:11:19.43+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
6156,0305K5F1P1R1,Ok,r03c05f01p01-ch1sk5fk1fl1.tiff,3,5,1,1,4,1,1,...,0,0.135623306,14398.233,2021-04-16T23:11:16.843+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105948,0305K71F1P1R1,Ok,r03c05f01p01-ch1sk71fk1fl1.tiff,3,5,1,1,70,1,1,...,0,0.135623097,251997.913,2021-04-19T17:11:16.523+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
107460,0305K72F1P1R1,Ok,r03c05f01p01-ch1sk72fk1fl1.tiff,3,5,1,1,71,1,1,...,0,0.135624498,255597.947,2021-04-19T18:11:16.557+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
108972,0305K73F1P1R1,Ok,r03c05f01p01-ch1sk73fk1fl1.tiff,3,5,1,1,72,1,1,...,0,0.1356242,259199.197,2021-04-19T19:11:17.807+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
110484,0305K74F1P1R1,Ok,r03c05f01p01-ch1sk74fk1fl1.tiff,3,5,1,1,73,1,1,...,0,0.135624796,262798.56,2021-04-19T20:11:17.17+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


In [16]:
dts = list()
for n, row in enumerate(position_df.iterrows()):
    if n == 0:
        t0 = row[1]['AbsTime']
        t0 = datetime.strptime(str(t0), f"%Y-%m-%dT%H:%M:%S.%f+{time_zone}")
    t1 = row[1]['AbsTime']
    t1 = datetime.strptime(str(t1), f"%Y-%m-%dT%H:%M:%S.%f+{time_zone}")
    dt = t1 - t0
    dt = dt.seconds/3600
    dts.append(dt)
    t0 = t1
    if n == len(position_df):
        break

### Print all dt in minutes between frames

In [17]:
[dt*60 for dt in dts]

[0.0,
 59.96666666666667,
 60.0,
 60.016666666666666,
 59.949999999999996,
 60.016666666666666,
 60.0,
 59.916666666666664,
 60.03333333333334,
 60.016666666666666,
 60.18333333333334,
 59.81666666666666,
 60.0,
 59.983333333333334,
 59.983333333333334,
 60.0,
 60.05,
 59.93333333333334,
 60.116666666666674,
 59.833333333333336,
 60.0,
 59.983333333333334,
 60.0,
 60.016666666666666,
 60.0,
 60.03333333333334,
 60.06666666666666,
 59.88333333333333,
 59.983333333333334,
 60.0,
 59.96666666666667,
 60.016666666666666,
 60.0,
 60.133333333333326,
 59.81666666666666,
 59.983333333333334,
 60.0,
 60.0,
 60.016666666666666,
 60.0,
 60.166666666666664,
 59.766666666666666,
 60.05,
 59.96666666666667,
 59.983333333333334,
 59.983333333333334,
 60.05,
 59.949999999999996,
 59.983333333333334,
 59.983333333333334,
 60.0,
 59.96666666666667,
 60.03333333333334,
 60.0,
 60.016666666666666,
 59.93333333333334,
 60.03333333333334,
 59.96666666666667,
 59.96666666666667,
 60.03333333333334,
 59.9833

In [21]:
print(f'Avg dt in hours:{np.mean(dts)}, and in minutes: {np.mean(dts)*60}')

Avg dt in hours:0.9865444444444443, and in minutes: 59.19266666666666


for all intent and purposes , dt = 1hr

## spatial scale

In [22]:
position_df['ImageResolutionX']

108       1.4949402023919043E-07
1620      1.4949402023919043E-07
3132      1.4949402023919043E-07
4644      1.4949402023919043E-07
6156      1.4949402023919043E-07
                   ...          
105948    1.4949402023919043E-07
107460    1.4949402023919043E-07
108972    1.4949402023919043E-07
110484    1.4949402023919043E-07
111996    1.4949402023919043E-07
Name: ImageResolutionX, Length: 75, dtype: object

In [23]:
position_df['ImageResolutionY']

108       1.4949402023919043E-07
1620      1.4949402023919043E-07
3132      1.4949402023919043E-07
4644      1.4949402023919043E-07
6156      1.4949402023919043E-07
                   ...          
105948    1.4949402023919043E-07
107460    1.4949402023919043E-07
108972    1.4949402023919043E-07
110484    1.4949402023919043E-07
111996    1.4949402023919043E-07
Name: ImageResolutionY, Length: 75, dtype: object

In [19]:
float(position_df['ImageResolutionX'].iloc[0])

1.4949402023919043e-07

In [24]:
x_scale = float(position_df['ImageResolutionX'].iloc[0])
y_scale = float(position_df['ImageResolutionY'].iloc[0])

In [26]:
scale = [x_scale, y_scale]
scale

[1.4949402023919043e-07, 1.4949402023919043e-07]

### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [27]:
metadata_path = '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml'
assay_layout = utils.read_harmony_metadata(metadata_path, assay_layout=True, mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...


/home/dayn/analysis/octopuslite-reader/octopuslite/utils.py:325: FutureWarning: The default value of regex will change from True to False in a future version.
  corresponding_mask_fns = input_img_fns.str.replace('ch(\d+)', 'ch99')
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:948: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


Extracting metadata complete!


Strain Compound Concentration ConcentrationEC  \
3 4    RD1     CTRL             0             EC0   
  5     WT     CTRL             0             EC0   
  6     WT      PZA            60            EC50   
  7     WT      RIF           0.1            EC50   
  8     WT      INH          0.04            EC50   
  9     WT      BDQ          0.02            EC50   
4 4    RD1     CTRL             0             EC0   
  5     WT     CTRL             0             EC0   
  6     WT      PZA            60            EC50   
  7     WT      RIF           0.1            EC50   
  8     WT      INH          0.04            EC50   
  9     WT      BDQ          0.02            EC50   
5 4    RD1     CTRL             0             EC0   
  5     WT     CTRL             0             EC0   
  6     WT      PZA           400            EC99   
  7     WT      RIF             2            EC99   
  8     WT      INH             2            EC99   
  9     WT      BDQ           2.5            EC99   
6 4    RD1     CTRL             0             EC0   
  5     WT     CTRL             0             EC0   
  6     WT      PZA           400            EC99   
  7     WT      RIF             2            EC99   
  8     WT      INH             2            EC99   
  9     WT      BDQ           2.5            EC99   

                                         Missing masks  
3 4                                               None  
  5  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  6  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  7                                               None  
  8  (1098, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  9  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
4 4  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  5  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  6                                               None  
  7  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  8  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  9                                               None  
5 4                                               None  
  5                                               None  
  6  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  7  (225, [/mnt/DATA/sandbox/pierre_live_cell_data...  
  8                                               None  
  9  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
6 4  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  5  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  6                                               None  
  7  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  8  (2025, [/mnt/DATA/sandbox/pierre_live_cell_dat...  
  9                                               None

### Define row and column of choice

In [28]:
row = '3'
column = '5'

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [29]:
%%time
images = tile.compile_mosaic(image_dir, metadata, row, column, set_plane=1, set_time=1)#tile.compile_mosaic(image_dir, metadata, row, column,).compute().compute()
images

CPU times: user 14.7 ms, sys: 3.54 ms, total: 18.3 ms
Wall time: 16.3 ms


dask.array<reshape, shape=(1, 2, 1, 6048, 6048), dtype=uint16, chunksize=(1, 1, 1, 2016, 2016), chunktype=numpy.ndarray>

# Launch napari image viewer

First to test the size of text overlays etc, then to create animations

In [36]:
def update_slider(event):
    # only trigger if update comes from first axis (optional)
        #ind_lambda = viewer.dims.indices[0]
    time = viewer.dims.current_step[0]
    viewer.text_overlay.text = f"{time:1.1f} hours"
text_size = 18
contrast_limits = [[100, 6000], [0, 3000]]

In [34]:
from napari_animation import Animation
from tqdm.auto import tqdm
import os

In [81]:
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column,
                             set_plane='sum_proj')

In [39]:
viewer = napari.Viewer()
viewer.add_image(images, 
                         channel_axis=1,
                         name=["macrophage", "mtb"],
                         colormap=["green", "magenta"],# "magenta"],
                         contrast_limits=contrast_limits, 
                         scale=scale
                         )
viewer.scale_bar.visible = True
viewer.scale_bar.unit = 'm'
viewer.scale_bar.font_size = text_size
viewer.text_overlay.visible = True
viewer.text_overlay.color = 'white'
viewer.text_overlay.position = 'bottom_left'
viewer.text_overlay.font_size = text_size
viewer.dims.events.current_step.connect(update_slider)

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


<function __main__.update_slider(event)>

In [43]:
cam_coords = viewer.camera.center
zoom = viewer.camera.zoom

### Define output path for images

In [12]:
output_dir = '/home/dayn/Videos/tb_mp4s/tiling/image_sequence'

In [54]:
plane = 'sum_proj'
for n, row in tqdm(enumerate(assay_layout.iterrows()), total = len(assay_layout), desc = 'Progress through positions'):
    print(n)
    if n != 19:
        continue    
    strain = row[1]['Strain']
    compound = row[1]['Compound']
    conc = row[1]['Concentration']
    name = f'{strain, compound, conc}'
    row, column = row[0] 
    
    fn = f'/home/dayn/Videos/tb_mp4s/tiling/z={plane}/{row,column,strain,compound,conc}[row,col,strain,compound,conc.].mp4'

    images = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, column,
                                 set_plane=plane)
    images = images.compute().compute()

    contrast_limits = [[100, 6000], [0, 3000]]
    viewer = napari.Viewer()
    viewer.add_image(images, 
                     channel_axis=1,
                     name=["macrophage", "mtb"],
                     colormap=["green", "magenta"],# "magenta"],
                     contrast_limits=contrast_limits, 
                     scale=microns_per_pixel
                     )

    viewer.scale_bar.visible = True
    viewer.scale_bar.unit = 'm'
    viewer.scale_bar.font_size = text_size
    viewer.text_overlay.visible = True
    viewer.text_overlay.color = 'white'
    viewer.text_overlay.position = 'bottom_left'
    viewer.text_overlay.font_size = text_size
    viewer.dims.events.current_step.connect(update_slider)
    animation = Animation(viewer)
    viewer.update_console({'animation': animation})
    # viewer.camera.center = (0, 0, 3024, 3024)
    viewer.dims.current_step = (0.0, cam_coords[-2], cam_coords[-1])
    viewer.camera.zoom = zoom#*0.97
    animation.capture_keyframe(steps = 100)
    viewer.dims.current_step = (74.0,  cam_coords[-2], cam_coords[-1])
    animation.capture_keyframe(steps = 100)

    animation.animate(fn, 
                      canvas_only=True,
                      fps = 7,
                      quality = 9)
    viewer.close()
    
#     ### and save out images
#     for t in tqdm(range(0, len(images)), leave = False, desc = 'Saving images'):
#         ### load frame into memory -- wont need this as already loaded
#         frame = images[t]#.compute().compute().astype(np.uint16)
#         ### convert to 8bit
#         frame = (frame/256).astype(np.uint8)
#         ### save out
#         for channel in [0,1]:
#             ### split channels
#             img = frame[channel]
#             ### create proper ch enumeration
#             channel = channel + 1
#             ### filenames
#             fn =  f't{t}_ch{channel}.tiff'
#             position_dir = f'strain:{strain}_compound:{compound}_conc:{conc}_row:{row}_column:{column}'
#             position_dir = os.path.join(output_dir, position_dir) 
#             if not os.path.exists(position_dir):
#                 os.mkdir(position_dir)
#             output_path = os.path.join(position_dir, fn)
#             cv2.imwrite(output_path, img)


Progress through positions:   0%|          | 0/24 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


Rendering frames...



  0%|                                                   | 0/101 [00:00<?, ?it/s]23-Jan-23 11:18:44 - imageio_ffmpeg - WARNING  - IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (706, 768) to (720, 768) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).

  1%|▍                                          | 1/101 [00:00<01:08,  1.46it/s][swscaler @ 0x5bf0000] Warning: data is not aligned! This can lead to a speed loss

100%|█████████████████████████████████████████| 101/101 [00:17<00:00,  5.62it/s]


20
21
22
23


In [53]:
for n, row in tqdm(enumerate(assay_layout.iterrows()), total = len(assay_layout), desc = 'Progress through positions'):
    
    if n == 0:
        continue
    strain = row[1]['Strain']
    compound = row[1]['Compound']
    conc = row[1]['Concentration']
    name = f'{strain, compound, conc}'
    row, column = row[0] 
    print(n, row, column)

Progress through positions:   0%|          | 0/24 [00:00<?, ?it/s]

1 3 5
2 3 6
3 3 7
4 3 8
5 3 9
6 4 4
7 4 5
8 4 6
9 4 7
10 4 8
11 4 9
12 5 4
13 5 5
14 5 6
15 5 7
16 5 8
17 5 9
18 6 4
19 6 5
20 6 6
21 6 7
22 6 8
23 6 9


In [56]:
for row in tqdm(assay_layout.iterrows(), total = len(assay_layout)):
    strain = row[1]['Strain']
    compound = row[1]['Compound']
    conc = row[1]['Concentration']
    name = f'{strain, compound, conc}'
    row, column = row[0]
    
    for plane in [2,3]:#'sum_proj']:        

        fn = f'/home/dayn/Videos/tb_mp4s/tiling/z={plane}/{row,column,strain,compound,conc}[row,col,strain,compound,conc.].mp4'
        
        if os.path.exists(fn):
            continue
        
        images = tile.compile_mosaic(image_dir, 
                                     metadata, 
                                     row, column,
                                     set_plane=plane)
        images = images.compute().compute()
        
        if plane == 'sum_proj':
            contrast_limits = [[100, 6000], [0, 3000]]
        else:
            contrast_limits = [[100, 2000], [0, 1000]]
        
        viewer = napari.Viewer()
        viewer.add_image(images, 
                         channel_axis=1,
                         name=["macrophage", "mtb"],
                         colormap=["green", "magenta"],# "magenta"],
                         contrast_limits=contrast_limits, 
                         scale=microns_per_pixel
                         )

        viewer.scale_bar.visible = True
        viewer.scale_bar.unit = 'm'
        viewer.scale_bar.font_size = text_size
        viewer.text_overlay.visible = True
        viewer.text_overlay.color = 'white'
        viewer.text_overlay.position = 'bottom_left'
        viewer.text_overlay.font_size = text_size
        viewer.dims.events.current_step.connect(update_slider)
        animation = Animation(viewer)
        viewer.update_console({'animation': animation})
        # viewer.camera.center = (0, 0, 3024, 3024)
        viewer.dims.current_step = (0.0, cam_coords[-2], cam_coords[-1])
        viewer.camera.zoom = zoom#*0.97
        animation.capture_keyframe(steps = 100)
        viewer.dims.current_step = (74.0,  cam_coords[-2], cam_coords[-1])
        animation.capture_keyframe(steps = 100)

        animation.animate(fn, 
                          canvas_only=True,
                          fps = 7,
                          quality = 9)
        viewer.close()

  0%|          | 0/24 [00:00<?, ?it/s]

In [237]:
import sys
sys.path.append('../macrohet/')
from notify import send_sms
send_sms('Video rendering complete')